# Convolutional Networks

### Load data

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn as skl

os.environ["CUDA_VISIBLE_DEVICES"]= "-1" # Force Tensorflow on CPU instead of GPU (seems to avoid an error with my CUDA compatible GPU)

C:\Users\lyeso\anaconda3\envs\exts-ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load training data form NPZ file

train_data = "train.npz"
X_tr = np.load(train_data)["features"]
y_tr = np.load(train_data)["labels"]
images_tr = np.load(train_data)["pixels"]
names_tr = np.load(train_data)["names"]

valid_data = "valid.npz"
X_val= np.load(valid_data)["features"]
y_val = np.load(valid_data)["labels"]
images_val = np.load(valid_data)["pixels"]
names_val = np.load(valid_data)["names"]

test_data = "test.npz"
X_te= np.load(test_data)["features"]
y_te = np.load(test_data)["labels"]
images_te = np.load(test_data)["pixels"]
names_te = np.load(test_data)["names"]

# Create a dictionnary for labels
labels_dict = {0: 'bike', 1 : 'car', 2: 'motorcycle', 3: 'other', 4:'truck', 5: 'van'}

In [18]:
images_tr[0].shape

(299, 299, 3)

### Let's use the train generator to have more images to train on and use in the Convolutional Network.

In [3]:
# Create image generator
train_generator = ImageDataGenerator(rescale=1/255, rotation_range = 10, horizontal_flip = True, vertical_flip = False, zoom_range = 0.2)
test_generator = ImageDataGenerator(rescale=1/255)

In [20]:
# Train, validation and test sets


# class_mode = categorical to have a one hot encoded "y" output.

trainset = train_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'train'), batch_size =500,
    shuffle=True, color_mode='rgb', class_mode = 'categorical') 

validset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'valid'),batch_size =500,
    shuffle=False, color_mode='rgb', class_mode = 'categorical')
testset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'test'),batch_size =500,
    shuffle=False,color_mode='rgb', class_mode = 'categorical')

Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


(280, 256, 256, 3)

### Let's create a 2 layers convolutional network

In [21]:
from tensorflow.keras.layers import Dropout

In [25]:
# Convolutional Network

model = keras.Sequential()


model.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=3, activation='relu', input_shape=(256, 256, 3)))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Conv2D(filters=16, kernel_size=2, strides=1, activation='relu'))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Flatten())
model.add(Dropout(rate=0.3)) # because of overfitting

model.add(keras.layers.Dense(units=trainset.num_classes, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 84, 84, 128)       9728      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 42, 42, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 64)        73792     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 16)          4112      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
__________

In [26]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [27]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [28]:
trainset.num_classes

6

In [29]:
# Train model
history = model.fit_generator(
    generator = trainset, 
    validation_data = validset, 
    epochs=50, 
    callbacks=[early_stopping],
    verbose = 1,
    use_multiprocessing=False
)

Epoch 1/50
1/1 [==============================] - 9s 9s/step - loss: 1.7971 - acc: 0.1750 - val_loss: 1.7806 - val_acc: 0.2230
Epoch 2/50
1/1 [==============================] - 6s 6s/step - loss: 1.7747 - acc: 0.2107 - val_loss: 1.7562 - val_acc: 0.2734
Epoch 3/50
1/1 [==============================] - 6s 6s/step - loss: 1.7789 - acc: 0.1857 - val_loss: 1.7477 - val_acc: 0.3165
Epoch 4/50
1/1 [==============================] - 6s 6s/step - loss: 1.7494 - acc: 0.2786 - val_loss: 1.7393 - val_acc: 0.3381
Epoch 5/50
1/1 [==============================] - 6s 6s/step - loss: 1.7744 - acc: 0.2179 - val_loss: 1.7376 - val_acc: 0.3237
Epoch 6/50
1/1 [==============================] - 6s 6s/step - loss: 1.7605 - acc: 0.2286 - val_loss: 1.7360 - val_acc: 0.3094
Epoch 7/50
1/1 [==============================] - 6s 6s/step - loss: 1.7572 - acc: 0.2607 - val_loss: 1.7318 - val_acc: 0.3741
Epoch 8/50
1/1 [==============================] - 6s 6s/step - loss: 1.7532 - acc: 0.2607 - val_loss: 1.7271 - 

### Let's try the same model with numpy arrays and change y with One Hot Encoding to implement in the model

In [11]:
### Let's use a bigger dataset for training.

images = np.concatenate((images_tr,images_val), axis=0, out=None) 
y = np.concatenate((y_tr,y_val), axis=0, out=None) 

In [37]:
from sklearn.preprocessing import OneHotEncoder as ohe

nominal_transformer = ohe(handle_unknown='ignore', sparse = False)

y_ohe = nominal_transformer.fit_transform(y.reshape(-1,1))
print(y_ohe.shape)


y_tr_ohe = nominal_transformer.transform(y_tr.reshape(-1,1))
print(y_tr_ohe.shape)

y_val_ohe = nominal_transformer.transform(y_val.reshape(-1,1))
print(y_val_ohe.shape)

(419, 6)
(280, 6)
(139, 6)


6

In [39]:
model2 = keras.Sequential()



model2.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=3, activation='relu', input_shape=(299, 299, 3)))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Conv2D(filters=16, kernel_size=2, strides=1, activation='relu'))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Flatten())
model2.add(Dropout(rate=0.3)) # because of overfitting

model2.add(keras.layers.Dense(units=trainset.num_classes, activation='softmax'))


model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 99, 99, 128)       9728      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)        73792     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 11, 11, 16)        4112      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 400)               0         
__________

In [40]:
# Compile the model
model2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc']) # loss = sparse_categorical_crossentropy does not need OHE

In [41]:


# Train model
history2 = model2.fit(
    x = images_tr, 
    y = y_tr_ohe, 
    epochs=50,
    validation_data = (images_val,y_val_ohe),
    callbacks=[early_stopping],
    verbose = 1,
    shuffle = True
)

Train on 280 samples, validate on 139 samples
Epoch 1/50
280/280 [==============================] - 6s 20ms/step - loss: 1.7695 - acc: 0.2000 - val_loss: 1.7340 - val_acc: 0.3022
Epoch 2/50
280/280 [==============================] - 5s 18ms/step - loss: 1.7471 - acc: 0.2250 - val_loss: 1.7388 - val_acc: 0.3094
Epoch 3/50
280/280 [==============================] - 5s 18ms/step - loss: 1.7451 - acc: 0.2286 - val_loss: 1.7379 - val_acc: 0.2158
Epoch 4/50
280/280 [==============================] - 5s 18ms/step - loss: 1.7116 - acc: 0.2786 - val_loss: 1.7303 - val_acc: 0.2734
Epoch 5/50
280/280 [==============================] - 5s 18ms/step - loss: 1.7340 - acc: 0.2714 - val_loss: 1.7216 - val_acc: 0.3309
Epoch 6/50
280/280 [==============================] - 5s 18ms/step - loss: 1.7097 - acc: 0.2750 - val_loss: 1.7097 - val_acc: 0.3597
Epoch 7/50
280/280 [==============================] - 6s 21ms/step - loss: 1.6757 - acc: 0.3464 - val_loss: 1.7016 - val_acc: 0.3597
Epoch 8/50
280/280 [===